In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

# ============ Positional Encoding ============
class PositionalEncoding(nn.Module):
    """Add positional information to embeddings"""
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


# ============ Key-Conditioned Multi-Head Attention ============
class KeyConditionedAttention(nn.Module):
    """
    Multi-head attention that is conditioned on encryption keys
    This ensures keys are DEEPLY integrated into the encryption
    """
    def __init__(self, d_model, num_heads=8):
        super(KeyConditionedAttention, self).__init__()

        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Query, Key, Value projections
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        # Key conditioning layer
        self.key_projection = nn.Linear(d_model, d_model)

        # Output projection
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(0.1)

    def forward(self, query, key, value, key_condition):
        """
        query, key, value: (batch, seq_len, d_model)
        key_condition: (batch, d_model) - encryption key features
        """
        batch_size = query.size(0)

        # Project encryption key
        key_features = self.key_projection(key_condition).unsqueeze(1)  # (batch, 1, d_model)

        # Add key conditioning to queries (this makes attention key-dependent)
        query = query + key_features

        # Multi-head projections
        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        # Apply attention to values
        context = torch.matmul(attn, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Output projection
        output = self.W_o(context)

        return output


# ============ Encoder Block with Key Conditioning ============
class EncoderBlock(nn.Module):
    """Transformer encoder block with deep key integration"""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderBlock, self).__init__()

        # Key-conditioned self-attention
        self.self_attn = KeyConditionedAttention(d_model, num_heads)

        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )

        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Key mixing layer
        self.key_mixer = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Linear(d_model, d_model)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, key_features):
        """
        x: (batch, seq_len, d_model)
        key_features: (batch, d_model)
        """
        # Self-attention with key conditioning
        attn_out = self.self_attn(x, x, x, key_features)
        x = self.norm1(x + self.dropout(attn_out))

        # Mix with key features
        batch_size, seq_len, _ = x.size()
        key_expanded = key_features.unsqueeze(1).expand(-1, seq_len, -1)
        x_with_key = torch.cat([x, key_expanded], dim=-1)
        key_mixed = self.key_mixer(x_with_key)
        x = x + key_mixed

        # Feed-forward
        ffn_out = self.ffn(x)
        x = self.norm2(x + ffn_out)

        return x


# ============ Enhanced Alice Encoder ============
class EnhancedAliceEncoder(nn.Module):
    """
    Alice: Encrypts string messages using transformer architecture
    with deep key conditioning
    """
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=6, d_ff=1024, max_len=512, dropout=0.1):
        super(EnhancedAliceEncoder, self).__init__()

        self.d_model = d_model
        self.vocab_size = vocab_size

        # Embedding layer for characters
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)

        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model, max_len)

        # Key encoder - processes encryption keys
        self.key_encoder = nn.Sequential(
            nn.Linear(512, d_model * 2),
            nn.LayerNorm(d_model * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model),
            nn.LayerNorm(d_model),
            nn.Tanh()
        )

        # Stack of encoder blocks
        self.encoder_blocks = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Output projection to encrypted sequence
        self.output_projection = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model),
            nn.Tanh()
        )

        self.dropout = nn.Dropout(dropout)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, message_tokens, key_sequences):
        """
        message_tokens: (batch, seq_len) - integer tokens
        key_sequences: list of 8 numpy arrays or tensors
        """
        batch_size, seq_len = message_tokens.size()
        device = message_tokens.device

        # Combine all key sequences into a single feature vector
        if isinstance(key_sequences[0], np.ndarray):
            key_combined = np.concatenate([ks[:512] for ks in key_sequences[:1]], axis=0)  # Use first key
            key_tensor = torch.FloatTensor(key_combined).to(device)
        else:
            key_combined = torch.cat([ks[:512] for ks in key_sequences[:1]], dim=0).to(device)
            key_tensor = key_combined

        # Expand for batch
        key_tensor = key_tensor.unsqueeze(0).expand(batch_size, -1)

        # Encode keys
        key_features = self.key_encoder(key_tensor)  # (batch, d_model)

        # Embed message tokens
        x = self.embedding(message_tokens) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # Pass through encoder blocks with key conditioning
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x, key_features)

        # Project to encrypted representation
        encrypted = self.output_projection(x)

        return encrypted


# ============ Alice Encryption Module ============
class AliceStringEncryption:
    """Wrapper for Alice encoder with string support"""
    def __init__(self, vocab_size=256, d_model=256, num_heads=8,
                 num_layers=6, max_len=512,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.alice = EnhancedAliceEncoder(
            vocab_size=vocab_size,
            d_model=d_model,
            num_heads=num_heads,
            num_layers=num_layers,
            max_len=max_len
        ).to(device)

        from part1_enhanced_dcgan import StringPreprocessor
        self.preprocessor = StringPreprocessor(max_length=max_len)

    def encrypt_string(self, message_string, key_sequences):
        """
        Encrypt a string message
        message_string: str or list of str
        key_sequences: from DCGAN key generator
        """
        # Convert string to tensor
        if isinstance(message_string, str):
            message_tensor = self.preprocessor.string_to_tensor(message_string).unsqueeze(0)
        else:
            message_tensor = self.preprocessor.batch_strings_to_tensor(message_string)

        message_tensor = message_tensor.to(self.device)

        # Encrypt
        self.alice.eval()
        with torch.no_grad():
            encrypted = self.alice(message_tensor, key_sequences)

        return encrypted

    def encrypt_tensor(self, message_tokens, key_sequences):
        """Encrypt already tokenized message"""
        self.alice.eval()
        with torch.no_grad():
            encrypted = self.alice(message_tokens, key_sequences)
        return encrypted


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("ENHANCED ALICE ENCODER - STRING ENCRYPTION")
    print("="*70)

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    alice_enc = AliceStringEncryption(device=device)

    # Test message
    test_message = "Hello World! This is a secret message."
    print(f"\nOriginal message: '{test_message}'")

    # Generate keys (simulated)
    key_sequences = [np.random.rand(512) for _ in range(8)]

    # Encrypt
    print("Encrypting...")
    encrypted = alice_enc.encrypt_string(test_message, key_sequences)

    print(f"\nEncryption complete!")
    print(f"  Input length: {len(test_message)} characters")
    print(f"  Encrypted shape: {encrypted.shape}")
    print(f"  Encrypted tensor (first 10 values):")
    print(f"  {encrypted[0, 0, :10]}")

    print("\n" + "="*70)

ENHANCED ALICE ENCODER - STRING ENCRYPTION

Original message: 'Hello World! This is a secret message.'
Encrypting...

Encryption complete!
  Input length: 38 characters
  Encrypted shape: torch.Size([1, 512, 256])
  Encrypted tensor (first 10 values):
  tensor([ 0.2025, -0.2799,  0.6672,  0.5963,  0.4357,  0.6974,  0.2273, -0.4616,
         0.4486,  0.6217])

